In [1]:
# Load packages and check versions
import sys
import numpy as np
import pandas as pd
import matplotlib as mpl
mpl.rcParams['figure.dpi'] = 400 #high res figures
import matplotlib.pyplot as plt
%matplotlib inline
import graphviz
import sklearn
import xgboost as xgb
import shap

print('The Python version is {}.\n'.format(sys.version))
print('The Numpy version is {}.\n'.format(np.__version__))
print('The Pandas version is {}.\n'.format(pd.__version__))
print('The Matplotlib version is {}.\n'.format(mpl.__version__))
print('The Graphviz version is {}.\n'.format(graphviz.__version__))
print('The Scikit-Learn version is {}.\n'.format(sklearn.__version__))
print('The XGBoost version is {}.\n'.format(xgb.__version__))
print('The Shap version is {}.\n'.format(shap.__version__))
# The Python version is 3.8.2 (default, Mar 26 2020, 10:43:30) 
# [Clang 4.0.1 (tags/RELEASE_401/final)].

# The Numpy version is 1.19.2.

# The Pandas version is 1.2.1.

# The Matplotlib version is 3.3.2.

# The Graphviz version is 0.15.

# The Scikit-Learn version is 0.23.2.

# The XGBoost version is 1.3.0.

# The Shap version is 0.37.0.

The Python version is 3.8.2 (default, Mar 26 2020, 10:43:30) 
[Clang 4.0.1 (tags/RELEASE_401/final)].

The Numpy version is 1.19.2.

The Pandas version is 1.2.1.

The Matplotlib version is 3.3.2.

The Graphviz version is 0.15.

The Scikit-Learn version is 0.23.2.

The XGBoost version is 1.3.0.

The Shap version is 0.37.0.



Maybe do an example on synthetic dataset (or some other real data from the UCI repository), as the illustration for the chapter, before this which would probably be the Activity. How to introduce the idea of early stopping? This seems somewhat complicated to combine with the 4 fold CV splits I've been using to compare different approaches. Maybe I could do it in the chapter but then not in the activity.

Docs for python: https://xgboost.readthedocs.io/en/latest/python/python_api.html#module-xgboost.sklearn

Maybe also worth paying attention to:
https://blog.cambridgespark.com/hyperparameter-tuning-in-xgboost-4ff9100a3b2f

---
Goals:

- Figure out how to correctly use the `grow_policy` paramter to experiment with the `lossguide` policy. I think I need to set the `tree_method` based on this, so will need to understand that hyperparameter as well. Described here and good to read through this thoroughly: https://xgboost.readthedocs.io/en/latest/parameter.html
- Is there some way to set the response variable fraction so it doesn't assume 0.5? I remember John talking about this at some point... probably good to ask him. Maybe good in general to ask him about these two goals in our 1-1 as I think I initially heard about both options from him.


Do I include all hyperparameter options in the book? I did this in the first edition and going through for the second edition it seems kind of like a waste of space... I'm wondering what the value add is above having readers just look at the website documentation. May want to go back and delete.

In [2]:
df = pd.read_csv('../Data/Chapter_1_cleaned_data.csv')

In [3]:
features_response = df.columns.tolist()

In [4]:
items_to_remove = ['ID', 'SEX', 'PAY_2', 'PAY_3', 'PAY_4', 'PAY_5', 'PAY_6',
                   'EDUCATION_CAT', 'graduate school', 'high school', 'none',
                   'others', 'university']

In [5]:
features_response = [item for item in features_response if item not in items_to_remove]
features_response

['LIMIT_BAL',
 'EDUCATION',
 'MARRIAGE',
 'AGE',
 'PAY_1',
 'BILL_AMT1',
 'BILL_AMT2',
 'BILL_AMT3',
 'BILL_AMT4',
 'BILL_AMT5',
 'BILL_AMT6',
 'PAY_AMT1',
 'PAY_AMT2',
 'PAY_AMT3',
 'PAY_AMT4',
 'PAY_AMT5',
 'PAY_AMT6',
 'default payment next month']

In [6]:
from sklearn.model_selection import train_test_split
from sklearn import tree

In [7]:
X_train, X_test, y_train, y_test = \
train_test_split(df[features_response[:-1]].values,
                 df['default payment next month'].values,
                 test_size=0.2, random_state=24)

In [8]:
# xgb_model = xgb.XGBClassifier(
#     n_estimators=1000,
#     max_depth=3,
#     learning_rate=0.01,
#     verbosity=1,
#     objective='binary:logistic',
#     use_label_encoder=False,
#     n_jobs=-1,
#     min_child_weight=100)
# params = {'max_depth':[6],
#           'n_estimators':[1000],
#           'colsample_bytree':[0.9],
#           'subsample':[0.9]}
# #Above can beat the RF in the 4-fold CV
# # 0.777226

xgb_model = xgb.XGBClassifier(
    n_estimators=1000,
    max_depth=0,
    max_leaves=100,
    learning_rate=0.01,
    verbosity=1,
    objective='binary:logistic',
    use_label_encoder=False,
    n_jobs=-1,
    min_child_weight=100,
    tree_method='hist',
    grow_policy='lossguide')
params = {'n_estimators':[1000],
          'colsample_bytree':[0.9],
          'subsample':[0.9]}
# 0.777415 so comparable, and works a lot faster than the above



In [9]:
from sklearn.model_selection import GridSearchCV

In [10]:
cv_xgb = GridSearchCV(xgb_model, param_grid=params, scoring='roc_auc',
                  n_jobs=None, refit=True, cv=4, verbose=1,
                  pre_dispatch=None, error_score=np.nan,
                  return_train_score=True)

In [11]:
%time cv_xgb.fit(X_train, y_train)

Fitting 4 folds for each of 1 candidates, totalling 4 fits
[13:34:13] WARNING: /Users/runner/miniforge3/conda-bld/xgboost_1607604592557/work/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[13:34:18] WARNING: /Users/runner/miniforge3/conda-bld/xgboost_1607604592557/work/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[13:34:22] WARNING: /Users/runner/miniforge3/conda-bld/xgboost_1607604592557/work/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[13:34:26] WARNING: /Users/runner/miniforge3/conda-bld/xgboost_1607604592557/work/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:   17.2s finished


[13:34:30] WARNING: /Users/runner/miniforge3/conda-bld/xgboost_1607604592557/work/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
CPU times: user 1min 12s, sys: 2.1 s, total: 1min 14s
Wall time: 22.5 s


GridSearchCV(cv=4,
             estimator=XGBClassifier(base_score=None, booster=None,
                                     colsample_bylevel=None,
                                     colsample_bynode=None,
                                     colsample_bytree=None, gamma=None,
                                     gpu_id=None, grow_policy='lossguide',
                                     importance_type='gain',
                                     interaction_constraints=None,
                                     learning_rate=0.01, max_delta_step=None,
                                     max_depth=0, max_leaves=100,
                                     min_child_weight=100, missing=nan,
                                     monotone_const...
                                     n_estimators=1000, n_jobs=-1,
                                     num_parallel_tree=None, random_state=None,
                                     reg_alpha=None, reg_lambda=None,
                             

In [12]:
xgb_cv_results_df = pd.DataFrame(cv_xgb.cv_results_)

In [13]:
xgb_cv_results_df

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_colsample_bytree,param_n_estimators,param_subsample,params,split0_test_score,split1_test_score,...,split3_test_score,mean_test_score,std_test_score,rank_test_score,split0_train_score,split1_train_score,split2_train_score,split3_train_score,mean_train_score,std_train_score
0,4.026787,0.111627,0.071355,0.001617,0.9,1000,0.9,"{'colsample_bytree': 0.9, 'n_estimators': 1000...",0.771406,0.769602,...,0.778443,0.777415,0.008091,1,0.824991,0.827054,0.82156,0.825088,0.824673,0.001977


Shaps

In [14]:
%time explainer = shap.TreeExplainer(cv_xgb)

Exception: Model type not yet supported by TreeExplainer: <class 'sklearn.model_selection._search.GridSearchCV'>